In [ ]:
# !pip install -U langgraph "langchain[anthropic]"

In [1]:
from pprint import pprint

In [ ]:
# to check the current Python environment
import sys
print(sys.prefix)

import os
print("LANGCHAIN_TRACING_V2:", os.environ.get("LANGCHAIN_TRACING_V2"))

i:\PROGRAMMING PROJECTS\langchain_crash_course\RestaurantNameGenerator\myenv


In [3]:
from dotenv import load_dotenv
import os

# Load variables from .env into environment
load_dotenv()


True

In [4]:
import os
assert "ANTHROPIC_API_KEY" in os.environ
assert "OPENAI_API_KEY" in os.environ


In [9]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [12]:
print(multiply.invoke({"a": 6, "b": 7}))  # returns 42
tool_call = {
    "type": "tool_call",
    "id": "1",
    "args": {"a": 42, "b": 7}
}
multiply.invoke(tool_call)

42


ToolMessage(content='294', name='multiply', tool_call_id='1')

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
from pprint import pprint

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# model = ChatAnthropic(
#     model="openai:gpt-4o",  # or "claude-3-7-sonnet-20240620" if supported
#     temperature=0
# )

agent = create_react_agent(
    # model='openai:gpt-4o',
    model='claude-3-5-haiku-latest',
    tools=[multiply, get_weather],
    prompt="You are a helpful assistant"
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "how is climate in berlin?"}]}
)
pprint(response)


OverloadedError: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load environment variables
load_dotenv()

# Confirm values
print("LangSmith Project:", os.environ.get("LANGCHAIN_PROJECT"))
print("LangSmith Tracing:", os.environ.get("LANGCHAIN_TRACING_V2"))
print("LangSmith API Key:", os.environ.get("LANGCHAIN_API_KEY")[:8], "...")  # Just to check if set

# Run LLM call
llm = ChatOpenAI(model="gpt-4o")
response = llm.invoke("Hello, LangSmith!")
print("Response:", response)


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.runnables import RunnableConfig  # required for metadata/tracing

def get_weather(city: str) -> str:  
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model="openai:gpt-4o",
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

# Add tracing metadata via config
config = RunnableConfig(configurable={"run_name": "weather_story_test"})

response = agent.invoke(
    {"messages": [{"role": "user", "content": "write 20 word story?"}]},
    config=config
)
from pprint import pprint
pprint(response)


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig

llm = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_template("Write a 20 word story.")
chain = prompt | llm

response = chain.invoke({}, config=RunnableConfig(configurable={"run_name": "basic_story_run"}))
print(response.content)


In [ ]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig
from langchain_core.output_parsers import StrOutputParser

# Load env
load_dotenv()

# Define LLM and prompt
llm = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_template("Write a 20 word story.")

# Define the full chain (this is required for tracing)
chain = prompt | llm | StrOutputParser()

# Add trace config
config = RunnableConfig(configurable={"run_name": "traceable_20_word_story"})

# Run
response = chain.invoke({}, config=config)
print("Story:", response)


In [ ]:
from langsmith.client import Client
from dotenv import load_dotenv

load_dotenv()

# You can also pass project name from your .env
client = Client()

# If you already set LANGCHAIN_PROJECT in your .env, use it
# project_name = "default"

project = client.read_project(project_name=project_name)
print(f"✅ LangSmith connection successful. Project: {project.name}")
